In [2]:
import pandas as pd
import numpy as np
import surprise
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold
import os 
os.chdir("D:/Training/Academy/ML(Python)/20. Recommender Systems/ml-100k")

In [3]:
ratings = pd.read_csv("u.data",sep='\t',
                      header=None, names = ['uid','iid','rating','timestamp'])
ratings = ratings.drop('timestamp',axis=1)
ratings.head()

,uid,iid,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 1 and 5


In [5]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [6]:
algo = surprise.SVD()
output = algo.fit(data.build_full_trainset()) # Calculates expected rating for all the users

List of User IDs

In [7]:
ratings['uid'].unique()

array([196, 186,  22, 244, 166, 298, 115, 253, 305,   6,  62, 286, 200,
       210, 224, 303, 122, 194, 291, 234, 119, 167, 299, 308,  95,  38,
       102,  63, 160,  50, 301, 225, 290,  97, 157, 181, 278, 276,   7,
        10, 284, 201, 287, 246, 242, 249,  99, 178, 251,  81, 260,  25,
        59,  72,  87,  42, 292,  20,  13, 138,  60,  57, 223, 189, 243,
        92, 241, 254, 293, 127, 222, 267,  11,   8, 162, 279, 145,  28,
       135,  32,  90, 216, 250, 271, 265, 198, 168, 110,  58, 237,  94,
       128,  44, 264,  41,  82, 262, 174,  43,  84, 269, 259,  85, 213,
       121,  49, 155,  68, 172,  19, 268,   5,  80,  66,  18,  26, 130,
       256,   1,  56,  15, 207, 232,  52, 161, 148, 125,  83, 272, 151,
        54,  16,  91, 294, 229,  36,  70,  14, 295, 233, 214, 192, 100,
       307, 297, 193, 113, 275, 219, 218, 123, 158, 302,  23, 296,  33,
       154,  77, 270, 187, 170, 101, 184, 112, 133, 215,  69, 104, 240,
       144, 191,  61, 142, 177, 203,  21, 197, 134, 180, 236, 26

Expected rating for any specific user for a specific item:

In [8]:
pred = algo.predict(uid='20',iid='101')
print(pred.est)

3.52986


In [9]:
pred

Prediction(uid='20', iid='101', r_ui=None, est=3.52986, details={'was_impossible': False})

Total Items:

In [10]:
iids = ratings['iid'].unique()
print(iids)

[ 242  302  377 ... 1637 1630 1641]


The list of items rated by user 60:

In [11]:
u_iid = list(ratings[ratings['uid']==60]['iid'])
print("List of items rated by user 60:", u_iid)
print("No. of items rated by user {0}: {1}".format(60, len(u_iid)))

List of items rated by user 60: [427, 60, 430, 162, 208, 403, 1021, 525, 222, 604, 47, 616, 1020, 79, 699, 179, 474, 508, 178, 172, 183, 490, 810, 152, 151, 28, 751, 514, 1126, 485, 498, 1124, 633, 272, 671, 176, 194, 502, 729, 71, 489, 186, 480, 660, 493, 207, 98, 218, 420, 163, 82, 605, 61, 88, 143, 511, 673, 21, 523, 132, 492, 613, 180, 495, 216, 659, 197, 638, 499, 13, 200, 138, 161, 558, 507, 64, 505, 416, 234, 56, 606, 417, 77, 529, 478, 199, 1123, 443, 842, 524, 593, 618, 366, 1121, 97, 661, 181, 121, 144, 195, 385, 665, 128, 210, 510, 174, 8, 357, 494, 735, 435, 1050, 608, 173, 433, 405, 835, 59, 582, 95, 736, 153, 229, 418, 506, 160, 212, 168, 515, 491, 12, 166, 609, 637, 404, 73, 30, 629, 185, 528, 602, 327, 650, 205, 265, 641, 411, 429, 546, 275, 50, 799, 211, 705, 69, 230, 482, 434, 675, 227, 684, 7, 204, 215, 9, 393, 136, 745, 70, 378, 225, 755, 286, 513, 1125, 1060, 708, 615, 445, 479, 89, 617, 96, 131, 209, 23, 656, 134, 517, 501, 484, 237, 654, 496, 1122, 133, 175, 603,

List of the items not rated by user 60:

In [12]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print("Items not rated by 60 or those items for which the expected ratings are to be predicted:",iids_to_predict )

Items not rated by 60 or those items for which the expected ratings are to be predicted: [   1    2    3 ... 1680 1681 1682]


In [13]:
len(iids_to_predict)

1474

Extracting the estimated rating for iids_to_predict

In [14]:
testset = [[60,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)
exp_ratings = pd.DataFrame(predictions)[['iid','est']]
exp_ratings.sort_values('est', ascending=False).iloc[:10]

,iid,est
117,169,4.909831
307,408,4.881814
83,114,4.781481
764,963,4.688273
223,315,4.652132
125,189,4.647160
378,530,4.645784
479,657,4.628999
43,57,4.605451
472,647,4.593324


### Matrix Factorization with SVD

In [ ]:
param_grid = {'n_factors': [50, 100, 150, 200, 250],  
              'lr_all':np.linspace(0.002, 0.01, 4), 'reg_all': np.linspace(0.02, 0.1, 4)  }
param_grid

In [24]:
kfold = KFold(n_splits=5, random_state=25, shuffle=True)
gs = GridSearchCV(surprise.SVD, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [25]:
gs.fit(data)

Best Score & Paramter:

In [27]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.9134066469812974
{'n_factors': 150, 'lr_all': 0.01, 'reg_all': 0.07333333333333333}


In [28]:
pd.DataFrame(gs.cv_results)

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_factors,param_lr_all,param_reg_all
0,0.946545,0.947393,0.953986,0.944631,0.946182,0.947747,0.003245,57,0.749688,0.749926,...,0.002794,61,0.661632,0.006162,0.133810,0.060160,"{'n_factors': 50, 'lr_all': 0.002, 'reg_all': ...",50,0.002000,0.020000
1,0.945545,0.949501,0.955400,0.944995,0.946861,0.948460,0.003803,59,0.750117,0.752669,...,0.003176,62,0.617488,0.001525,0.140057,0.063844,"{'n_factors': 50, 'lr_all': 0.002, 'reg_all': ...",50,0.002000,0.046667
2,0.945042,0.948982,0.955082,0.945049,0.947303,0.948292,0.003705,58,0.750494,0.753263,...,0.003079,64,0.620786,0.006914,0.112371,0.052542,"{'n_factors': 50, 'lr_all': 0.002, 'reg_all': ...",50,0.002000,0.073333
3,0.945607,0.950095,0.955795,0.946175,0.947711,0.949076,0.003702,61,0.751441,0.754684,...,0.003073,69,0.615570,0.007162,0.133241,0.064489,"{'n_factors': 50, 'lr_all': 0.002, 'reg_all': ...",50,0.002000,0.100000
4,0.932199,0.933802,0.939903,0.934202,0.933837,0.934789,0.002649,40,0.734637,0.736254,...,0.001991,34,0.611612,0.006068,0.134326,0.060687,"{'n_factors': 50, 'lr_all': 0.0046666666666666...",50,0.004667,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.920792,0.924270,0.930661,0.921625,0.921314,0.923732,0.003667,25,0.729188,0.731943,...,0.003189,26,1.323978,0.062008,0.148721,0.061737,"{'n_factors': 250, 'lr_all': 0.007333333333333...",250,0.007333,0.100000
76,0.945577,0.949946,0.953893,0.942514,0.943226,0.947031,0.004303,56,0.744811,0.747120,...,0.003553,55,1.423801,0.120219,0.125922,0.045468,"{'n_factors': 250, 'lr_all': 0.01, 'reg_all': ...",250,0.010000,0.020000
77,0.921567,0.920762,0.929485,0.919757,0.917249,0.921764,0.004125,15,0.727270,0.724673,...,0.003291,13,1.288863,0.032695,0.138453,0.060369,"{'n_factors': 250, 'lr_all': 0.01, 'reg_all': ...",250,0.010000,0.046667
78,0.911549,0.915445,0.921372,0.913750,0.912856,0.914994,0.003431,3,0.720276,0.722460,...,0.002596,4,1.295378,0.021832,0.142044,0.063781,"{'n_factors': 250, 'lr_all': 0.01, 'reg_all': ...",250,0.010000,0.073333


Saving the best model in a file

In [ ]:
from surprise import dump
best_model = gs.best_estimator['rmse']
best_model.fit(data.build_full_trainset()) # Refit the best model on the full trainset
dump.dump('svd_ml_100k', predictions=None, algo=best_model)

Loading the saved model

In [37]:
loaded_obj = dump.load('svd_ml_100k')
bm = loaded_obj[1]

Recommendations for User 60

In [38]:
testset = [[60,iid,0.] for iid in iids_to_predict]
predictions = bm.test(testset)
exp_ratings = pd.DataFrame(predictions)[['iid','est']]
exp_ratings.sort_values('est', ascending=False).iloc[:10]

,iid,est
1240,1449,4.884996
117,169,4.856021
307,408,4.834382
83,114,4.784761
226,318,4.703786
985,1194,4.622842
370,512,4.592319
228,320,4.584520
125,189,4.569183
764,963,4.569012


In [44]:
movies = pd.read_csv("movies_list.csv",encoding='latin-1')
movies

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-95,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-95,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-95,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-95,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-95,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-98,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-98,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-98,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-94,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
movies = movies.iloc[:, :2]
movies.columns = ['iid','movie title']
movies.merge(exp_ratings, on='iid').sort_values('est', ascending=False).iloc[:5]

,iid,movie title,est
1240,1449,Pather Panchali (1955),4.884996
117,169,"Wrong Trousers, The (1993)",4.856021
307,408,"Close Shave, A (1995)",4.834382
83,114,Wallace & Gromit: The Best of Aardman Animatio...,4.784761
226,318,Schindler's List (1993),4.703786
